In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gcsfs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import io
import os
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch.optim as optim
import torch.nn as nn
from evaluate import load
import gc
import time
from google.colab import drive
from google.colab import auth
from google.cloud import storage

plt.style.use('default')
sns.set_palette("colorblind")

In [5]:
auth.authenticate_user()

In [6]:
class CXR(Dataset):
  def __init__(self, dataframe, processor, max_length):
    super().__init__()
    self.dataframe = dataframe.reset_index(drop=True)
    self.processor = processor
    self.max_length = max_length
    self.prompt = "List pathalogical findings for this chest X-ray:"
    self.storage_client = storage.Client(project='silken-physics-467815-g5')
  def __len__(self):
    return len(self.dataframe)

  def _loadImage(self,subject_id, study_id, dicom_id):
    try:
      bucket_name = "mimic-cxr-jpg-2.1.0.physionet.org"
      image_path = f"files/p{subject_id[:2]}/p{subject_id}/s{study_id}/{dicom_id}.jpg"
      bucket = self.storage_client.bucket(bucket_name, user_project='silken-physics-467815-g5')
      blob = bucket.blob(image_path)
      image_bytes = blob.download_as_bytes()
      image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
      return image
    except Exception as e:
      print(f"Error loading image {image_path}: {str(e)}")
      return None # Return None if image loading fails

  def __getitem__(self, index):
    row = self.dataframe.iloc[index]
    miniReport = str(row['mini_report'])
    subject = str(row['subject_id'])
    study = str(row['study_id'])
    dicom = str(row['dicom_id'])
    image = self._loadImage(subject_id=subject,study_id=study,dicom_id=dicom)
    inputs = self.processor(images=image, text=self.prompt, return_tensors="pt", padding="max_length",
                            truncation=True, max_length=self.max_length)
    labels = self.processor.tokenizer(miniReport, return_tensors="pt", padding="max_length",
                                      truncation=True, max_length=self.max_length)["input_ids"]

    return {
      "pixel_values": inputs["pixel_values"],  # Shape: [1, 3, H, W]
      "input_ids": inputs["input_ids"],        # Shape: [1, max_length]
      "attention_mask": inputs["attention_mask"],  # Shape: [1, max_length]
      "labels": labels                         # Shape: [1, max_length]
    }

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/git-base")

In [8]:
train_df =pd.read_csv('./train_split.csv')
test_df =pd.read_csv('./test_split.csv')
val_df=pd.read_csv('./val_split.csv')
max_length = max(len(processor.tokenizer.encode(report)) for report in train_df['mini_report'])
print(f"Max length of mini-reports: {max_length}")

Max length of mini-reports: 92


In [9]:
train_dataset = CXR(train_df, processor, max_length)
val_dataset = CXR(val_df, processor, max_length)
test_dataset = CXR(test_df, processor, max_length)

In [10]:
print(train_dataset[0]['pixel_values'].shape)
print(train_dataset[0]['input_ids'].shape)
print(train_dataset[0]['attention_mask'].shape)
print(train_dataset[0]['labels'].shape)


torch.Size([1, 3, 224, 224])
torch.Size([1, 92])
torch.Size([1, 92])
torch.Size([1, 92])


## Load GIT Model

In [ ]:
device = torch.device("cuda")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base", trust_remote_code=True)


In [12]:
memory_allocated = torch.cuda.memory_allocated() / (1024 * 1024)
print('Memory Allocated before loading GIT Large :',memory_allocated,'MB')

Memory Allocated before loading GIT Large : 0.0 MB


In [13]:
model.to(device)

GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
             

In [17]:
memory_allocated = torch.cuda.memory_allocated() / (1024 * 1024)
print('Memory Allocated after loading GIT Base :',memory_allocated,'MB')

Memory Allocated after loading GIT Base : 674.91845703125 MB


In [ ]:
import gc
del model
torch.cuda.empty_cache()
gc.collect()

## Dataloader helper

In [19]:
def collate_fn(batch):
    pixel_values = torch.cat([item["pixel_values"] for item in batch], dim=0)  # [batch_size, 3, H, W]
    input_ids = torch.cat([item["input_ids"] for item in batch], dim=0)        # [batch_size, max_length]
    attention_mask = torch.cat([item["attention_mask"] for item in batch], dim=0)  # [batch_size, max_length]
    labels = torch.cat([item["labels"] for item in batch], dim=0)               # [batch_size, max_length]

    #print(f"Batch shapes: pixel_values={pixel_values.shape}, input_ids={input_ids.shape}, "
    #      f"attention_mask={attention_mask.shape}, labels={labels.shape}")

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

## Training

In [20]:
import time

In [ ]:
# batch_size = 64
# learning_rate = 1e-5
# start_time = time.time()  # Record start time
# torch.cuda.empty_cache()
# gc.collect()
# # Initialize model
# model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
# model.to(device)
# print('------------------------------------------------------------------------------------------------------------------------------------------------')
# print(f'Hyperparameters are Batch Size = {batch_size}, Learning Rate = {learning_rate} with {model.num_parameters()} trainable parameters')
# print('------------------------------------------------------------------------------------------------------------------------------------------------')
# # Create dataloaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# results = []
# def make_train_step(model):
#     def train_step(batch):
#         model.train()
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         return loss.item()
#     return train_step

# def make_val_step(model):
#     def val_step(batch):
#         model.eval()
#         with torch.no_grad():
#             batch = {k: v.to(device) for k, v in batch.items()}
#             outputs = model(**batch)
#             loss = outputs.loss
#             return loss.item()
#     return val_step

# def for_epochs(epochs, train_loader, val_loader, train_step, val_step, scheduler, early_stop_window=2):
#     train_losses = []
#     val_losses = []
#     memory_usage = []
#     best_val_loss = float('inf')
#     best_weights = None
#     epochs_with_no_improvement = 0

#     for epoch in range(epochs):
#         # Training
#         train_loss = 0.0
#         train_samples = 0
#         for batch in train_loader:
#             loss = train_step(batch)
#             batch_size_actual = batch["pixel_values"].size(0)
#             train_loss += loss * batch_size_actual
#             train_samples += batch_size_actual
#             break
#         avg_train_loss = train_loss / train_samples
#         train_losses.append(avg_train_loss)


#         # Validation
#         val_loss = 0.0
#         val_samples = 0
#         with torch.no_grad():
#             for batch in val_loader:
#                 loss = val_step(batch)
#                 batch_size_actual = batch["pixel_values"].size(0)
#                 val_loss += loss * batch_size_actual
#                 val_samples += batch_size_actual
#         avg_val_loss = val_loss / val_samples
#         val_losses.append(avg_val_loss)

#         # Memory tracking
#         torch.cuda.synchronize()
#         memory_mb = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MB
#         memory_usage.append(memory_mb)

#         print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}, Memory: {memory_mb:.2f} MB")

#         # Scheduler and early stopping
#         scheduler.step(avg_val_loss)
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             torch.save(model.state_dict(), "best_weights.pt")
#             epochs_with_no_improvement = 0
#         else:
#             epochs_with_no_improvement += 1
#             if epochs_with_no_improvement >= early_stop_window:
#                 print("Early stopping triggered")
#     best_weights = torch.load("best_weights.pt")

#     training_time = time.time() - start_time
#     print(f"Total Training Time: {training_time:.4f} seconds")
#     return train_losses, val_losses, memory_usage, best_weights, training_time

# train_step = make_train_step(model)
# val_step = make_val_step(model)
# train_losses, val_losses, memory_usage, best_weights, training_time = for_epochs(5, train_loader, val_loader, train_step, val_step, scheduler)
# results.append({
#         "experiment": "fft",
#         "train_losses": train_losses,
#         "val_losses": val_losses,
#         "memory_usage_MB": memory_usage,
#         "training_time_seconds": training_time
#     })
# del model
# torch.cuda.empty_cache()
# gc.collect()

# folder_path = '/content/drive/MyDrive/GIT AblationStratergy/fft_ablationStudy/results'
# os.makedirs(folder_path, exist_ok=True)
# results_df = pd.DataFrame(results)
# results_df.to_csv(os.path.join(folder_path, 'ffttrain_target_modules_ablation_results.csv'), index=False)
# print("\n Fft Ablation Study Training Results (Target Modules):")
# print(results_df[["experiment", "test_loss", "bleu_score", "training_time_seconds", "trainable_parameters"]])

In [ ]:
del model
torch.cuda.empty_cache()
gc.collect()

In [26]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM
import time
import gc
import os
import pandas as pd
import pickle

# =============================================================================
# RESOURCE MANAGEMENT CONFIGURATION
# =============================================================================
EPOCHS_PER_CHUNK = 4  # Train 2 epochs per session, then restart
TOTAL_EPOCHS = 20
CHECKPOINT_DIR = '/content/drive/MyDrive/GIT_checkpoints/'
RESULTS_DIR = '/content/drive/MyDrive/GIT AblationStratergy/fft_ablationStudy/results'

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# =============================================================================
# CHECKPOINT MANAGEMENT FUNCTIONS
# =============================================================================
def save_checkpoint(model, optimizer, scheduler, epoch, train_losses, val_losses, memory_usage,
                   results, start_time, best_val_loss, epochs_with_no_improvement, best_weights=None):
    """Save complete training state"""

    # Save or reference best weights
    best_weights_path = os.path.join(CHECKPOINT_DIR, "best_weights.pt")
    if best_weights is not None:
        torch.save(best_weights, best_weights_path)

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_losses': train_losses,
        'val_losses': val_losses,
        'memory_usage': memory_usage,
        'results': results,
        'start_time': start_time,
        'best_val_loss': best_val_loss,
        'epochs_with_no_improvement': epochs_with_no_improvement,
        'best_weights_path': best_weights_path
    }

    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'checkpoint_epoch_{epoch}.pt')
    torch.save(checkpoint, checkpoint_path)
    print(f"✅ Checkpoint saved: {checkpoint_path}")
    return checkpoint_path

def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    """Load complete training state"""
    print(f"📂 Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location='cuda')

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    return checkpoint

def find_latest_checkpoint():
    """Find the most recent checkpoint"""
    checkpoints = [f for f in os.listdir(CHECKPOINT_DIR) if f.startswith('checkpoint_epoch_')]
    if not checkpoints:
        return None

    # Sort by epoch number
    checkpoints.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest = os.path.join(CHECKPOINT_DIR, checkpoints[-1])
    print(f"🔍 Found latest checkpoint: {latest}")
    return latest

# =============================================================================
# GPU PERFORMANCE CHECK
# =============================================================================
def quick_gpu_benchmark():
    """Quick GPU performance test"""
    print("🧪 Testing GPU performance...")
    if not torch.cuda.is_available():
        print("❌ CUDA not available!")
        return False

    device = torch.device('cuda')
    a = torch.randn(2000, 2000, device=device)
    b = torch.randn(2000, 2000, device=device)

    torch.cuda.synchronize()
    start_time = time.time()

    for _ in range(50):
        c = torch.mm(a, b)

    torch.cuda.synchronize()
    benchmark_time = time.time() - start_time

    print(f"GPU benchmark: {benchmark_time:.2f}s")
    print(f"GPU: {torch.cuda.get_device_name()}")

    # Clean up
    del a, b, c
    torch.cuda.empty_cache()

    if benchmark_time > 3.0:  # Should be ~1-2s on healthy A100
        print("⚠️  WARNING: GPU performance seems degraded!")
        return False
    else:
        print("✅ GPU performance looks good!")
        return True

# =============================================================================
# ENHANCED TRAINING FUNCTIONS
# =============================================================================
def make_train_step_with_monitoring(model):
    def train_step(batch):
        model.train()
        batch_start = time.time()

        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        batch_time = time.time() - batch_start
        if batch_time > 10.0:  # Warn if batch takes >10s
            print(f"⚠️  Slow batch: {batch_time:.2f}s")

        return loss.item()
    return train_step

def make_val_step_with_monitoring(model):
    def val_step(batch):
        model.eval()
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            return loss.item()
    return val_step

def train_chunk(start_epoch, end_epoch, model, train_loader, val_loader, optimizer,
                scheduler, train_losses, val_losses, memory_usage, best_val_loss,
                epochs_with_no_improvement, start_time):
    """Train for a chunk of epochs with monitoring"""

    train_step = make_train_step_with_monitoring(model)
    val_step = make_val_step_with_monitoring(model)

    print(f"🚀 Training epochs {start_epoch+1} to {end_epoch}")

    for epoch in range(start_epoch, end_epoch):
        epoch_start = time.time()
        print(f"\n--- Epoch {epoch+1}/{TOTAL_EPOCHS} ---")

        # Training
        train_loss = 0.0
        train_samples = 0
        batch_count = 0

        for batch_idx, batch in enumerate(train_loader):
            if batch_idx == 0:  # Only process first batch for testing
                loss = train_step(batch)
                batch_size_actual = batch["pixel_values"].size(0)
                train_loss += loss * batch_size_actual
                train_samples += batch_size_actual
                batch_count += 1
                print(f"  Batch {batch_idx+1} completed, loss: {loss:.6f}")
                break

        avg_train_loss = train_loss / train_samples if train_samples > 0 else 0
        train_losses.append(avg_train_loss)

        # Validation
        val_loss = 0.0
        val_samples = 0
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                loss = val_step(batch)
                batch_size_actual = batch["pixel_values"].size(0)
                val_loss += loss * batch_size_actual
                val_samples += batch_size_actual
                if batch_idx >= 4:  # Limit validation batches for speed
                    break

        avg_val_loss = val_loss / val_samples if val_samples > 0 else 0
        val_losses.append(avg_val_loss)

        # Memory tracking
        torch.cuda.synchronize()
        memory_mb = torch.cuda.memory_allocated() / (1024 * 1024)
        memory_usage.append(memory_mb)

        epoch_time = time.time() - epoch_start
        total_time = time.time() - start_time

        print(f"  ✅ Epoch {epoch+1} completed in {epoch_time:.1f}s")
        print(f"     Train Loss: {avg_train_loss:.6f}")
        print(f"     Val Loss: {avg_val_loss:.6f}")
        print(f"     Memory: {memory_mb:.0f} MB")
        print(f"     Total Time: {total_time:.1f}s")

        # Scheduler and early stopping
        scheduler.step(avg_val_loss)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_weights = model.state_dict()  # Store best weights
            torch.save(best_weights, os.path.join(CHECKPOINT_DIR, "best_weights.pt"))
            epochs_with_no_improvement = 0
            print(f"     🎯 New best validation loss: {best_val_loss:.6f}")
        else:
            epochs_with_no_improvement += 1
            best_weights = None  # No improvement this epoch

        # Save checkpoint after each epoch
        save_checkpoint(model, optimizer, scheduler, epoch + 1, train_losses,
                       val_losses, memory_usage, [], start_time, best_val_loss,
                       epochs_with_no_improvement, best_weights)

    return train_losses, val_losses, memory_usage, best_val_loss, epochs_with_no_improvement

# =============================================================================
# MAIN TRAINING ORCHESTRATOR
# =============================================================================
def run_resource_managed_training():
    """Main training function with resource management"""

    # Check GPU performance first
    if not quick_gpu_benchmark():
        print("🔄 Consider restarting runtime for better GPU performance")
        return

    # Initialize training parameters
    batch_size = 64
    learning_rate = 1e-5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Clear memory
    torch.cuda.empty_cache()
    gc.collect()

    # Check for existing checkpoint
    latest_checkpoint = find_latest_checkpoint()

    if latest_checkpoint:
        print("🔄 Resuming from checkpoint...")
        # Load model architecture first
        model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
        model.to(device)

        # Initialize optimizer and scheduler
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

        # Load checkpoint
        checkpoint = load_checkpoint(latest_checkpoint, model, optimizer, scheduler)

        # Restore training state
        start_epoch = checkpoint['epoch']
        train_losses = checkpoint['train_losses']
        val_losses = checkpoint['val_losses']
        memory_usage = checkpoint['memory_usage']
        start_time = checkpoint['start_time']
        best_val_loss = checkpoint['best_val_loss']
        epochs_with_no_improvement = checkpoint['epochs_with_no_improvement']

        print(f"📍 Resuming from epoch {start_epoch}")

    else:
        print("🆕 Starting fresh training...")
        # Initialize everything from scratch
        start_time = time.time()
        model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
        model.to(device)

        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

        start_epoch = 0
        train_losses = []
        val_losses = []
        memory_usage = []
        best_val_loss = float('inf')
        epochs_with_no_improvement = 0

    print('='*80)
    print(f'🎯 Training Configuration:')
    print(f'   Batch Size: {batch_size}')
    print(f'   Learning Rate: {learning_rate}')
    print(f'   Total Epochs: {TOTAL_EPOCHS}')
    print(f'   Epochs per Chunk: {EPOCHS_PER_CHUNK}')
    print(f'   Trainable Parameters: {model.num_parameters():,}')
    print('='*80)

    # DEFINE YOUR DATALOADERS HERE
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Calculate remaining epochs and chunks
    remaining_epochs = TOTAL_EPOCHS - start_epoch
    if remaining_epochs <= 0:
        print("✅ Training already completed!")
        return

    # Determine chunk size for this session
    epochs_this_session = min(EPOCHS_PER_CHUNK, remaining_epochs)
    end_epoch = start_epoch + epochs_this_session

    print(f"📊 This session: epochs {start_epoch+1} to {end_epoch}")
    print(f"💾 Results will be saved to: {RESULTS_DIR}")

    # Train the chunk
    try:
        train_losses, val_losses, memory_usage, best_val_loss, epochs_with_no_improvement = train_chunk(
            start_epoch, end_epoch, model, train_loader, val_loader, optimizer,
            scheduler, train_losses, val_losses, memory_usage, best_val_loss,
            epochs_with_no_improvement, start_time
        )

        print(f"\n✅ Chunk completed successfully!")
        print(f"   Epochs completed: {end_epoch}/{TOTAL_EPOCHS}")

        # If training complete, save final results
        if end_epoch >= TOTAL_EPOCHS:
            print("🎉 Training completed! Saving final results...")

            # Load best weights
            best_weights_path = os.path.join(CHECKPOINT_DIR, "best_weights.pt")
            if os.path.exists(best_weights_path):
                best_weights = torch.load(best_weights_path)
                print(f"✅ Best weights loaded from {best_weights_path}")
            else:
                best_weights = model.state_dict()
                print("⚠️  No best weights file found, using final model state")

            training_time = time.time() - start_time

            results = [{
                "experiment": "fft",
                "train_losses": train_losses,
                "val_losses": val_losses,
                "memory_usage_MB": memory_usage,
                "training_time_seconds": training_time,
                "best_val_loss": best_val_loss,
                "best_weights": best_weights  # Added missing best_weights
            }]

            # Save results
            results_df = pd.DataFrame(results)
            results_path = os.path.join(RESULTS_DIR, 'ffttrain_target_modules_ablation_results.csv')
            results_df.to_csv(results_path, index=False)

            print(f"💾 Final results saved to: {results_path}")
            print(f"⏱️  Total training time: {training_time:.1f} seconds")

        else:
            print(f"\n⏭️  Next steps:")
            print(f"   1. Runtime → Disconnect and delete runtime")
            print(f"   2. Wait 2-3 minutes")
            print(f"   3. Reconnect and run this code again")
            print(f"   4. Training will resume from epoch {end_epoch+1}")

    except Exception as e:
        print(f"❌ Error during training: {e}")
        print("💾 Checkpoint should be saved. You can resume later.")
        raise

    finally:
        # Cleanup
        del model
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 Memory cleaned up")

# =============================================================================
# USAGE INSTRUCTIONS
# =============================================================================
print("""
🚀 RESOURCE-MANAGED TRAINING SETUP
=====================================

BEFORE RUNNING:
1. Define your train_loader and val_loader
2. Ensure your dataset and collate_fn are ready
3. Update CHECKPOINT_DIR and RESULTS_DIR paths if needed

TO RUN:
run_resource_managed_training()

WORKFLOW:
- Trains 4 epochs per session
- Saves checkpoints automatically
- Disconnects → waits → reconnects for fresh GPU
- Resumes automatically from latest checkpoint
- Saves final results when complete

BENEFITS:
✅ Avoids degraded GPU instances
✅ Saves compute units with efficient training
✅ Automatic resume from failures
✅ Memory management built-in
✅ Performance monitoring
""")

# Uncomment to run (after adding your dataloaders):
run_resource_managed_training()


🚀 RESOURCE-MANAGED TRAINING SETUP

BEFORE RUNNING:
1. Define your train_loader and val_loader
2. Ensure your dataset and collate_fn are ready
3. Update CHECKPOINT_DIR and RESULTS_DIR paths if needed

TO RUN:
run_resource_managed_training()

WORKFLOW:
- Trains 4 epochs per session
- Saves checkpoints automatically  
- Disconnects → waits → reconnects for fresh GPU
- Resumes automatically from latest checkpoint
- Saves final results when complete

BENEFITS:
✅ Avoids degraded GPU instances
✅ Saves compute units with efficient training
✅ Automatic resume from failures
✅ Memory management built-in
✅ Performance monitoring

🧪 Testing GPU performance...
GPU benchmark: 0.06s
GPU: NVIDIA A100-SXM4-40GB
✅ GPU performance looks good!
🔍 Found latest checkpoint: /content/drive/MyDrive/GIT_checkpoints/checkpoint_epoch_6.pt
🔄 Resuming from checkpoint...
📂 Loading checkpoint: /content/drive/MyDrive/GIT_checkpoints/checkpoint_epoch_6.pt
📍 Resuming from epoch 6
🎯 Training Configuration:
   Batch Size: 

## Testing

In [ ]:
CHECKPOINT_DIR = '/content/drive/MyDrive/GIT_checkpoints/'
# Fix 1: Set tokenizer padding to left side for decoder-only models
processor.tokenizer.padding_side = 'left'
best_weights_path = os.path.join(CHECKPOINT_DIR, "best_weights.pt")
# Load and prepare model
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
model.load_state_dict(torch.load('best_weights.pt', weights_only=True))
model.to(device)
model.eval()

test_loader = DataLoader(test_dataset, batch_size=64, collate_fn=collate_fn)
test_loss = 0.0
test_samples = 0
predictions = []
references = []

results = []
bleu = load("bleu")

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        batch_size_actual = batch["pixel_values"].size(0)
        test_loss += loss.item() * batch_size_actual
        test_samples += batch_size_actual

        # Generate predictions
        pixel_values = batch["pixel_values"]
        input_ids = batch["input_ids"]

        # Fix 2: Use max_new_tokens instead of max_length to avoid length conflicts
        generated_ids = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            max_new_tokens=92,  # Generate up to 50 new tokens
            do_sample=False,    # Use greedy decoding for reproducible results
            pad_token_id=processor.tokenizer.pad_token_id
        )

        generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend([text.replace("List pathalogical findings for this chest X-ray:", "").strip() for text in generated_texts])

        # Decode labels for references
        label_ids = batch["labels"]
        reference_texts = processor.batch_decode(label_ids, skip_special_tokens=True)
        references.extend(reference_texts)
        break

avg_test_loss = test_loss / test_samples
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print(f"Test Results with Best Model:")
print(f"Test Loss: {avg_test_loss:.6f}, BLEU Score: {bleu_score['bleu']:.4f}")
results.append({
        "experiment": "fftTesting",
        "test_loss": test_loss,
        "bleu_score": bleu_score
    })

del model
torch.cuda.empty_cache()
gc.collect()
folder_path = '/content/drive/MyDrive/GIT AblationStratergy/fft_test_ablationStudy/results'
os.makedirs(folder_path, exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(folder_path, 'fftTest_target_modules_ablation_results.csv'), index=False)
print("\nFFT Ablation Study Testing Results (Target Modules):")
print(results_df[["experiment", "test_loss", "bleu_score", "training_time_seconds", "trainable_parameters"]])

In [4]:
drive.mount('/content/drive', force_remount=True)
plt.style.use('default')
sns.set_palette("colorblind")

Mounted at /content/drive


In [18]:
# Check what's actually running
import torch
print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.1f}GB")
print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.1f}GB")

# See GPU utilization
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu,power.draw', '--format=csv,noheader,nounits'], capture_output=True, text=True)
print(f"GPU Utilization: {result.stdout.strip()}")

GPU Memory Allocated: 2.8GB
GPU Memory Reserved: 33.3GB
GPU Utilization: 0, 54.05
